### Throughout this exercise, you may need to refer to online documentation.

Pandas: http://pandas.pydata.org/pandas-docs/stable/

Numpy & Scipy: https://docs.scipy.org/doc/

In [2]:
# First, import packages and load some data
import pandas      as pd
import numpy       as np
import scipy       as sp
import scipy.stats as stats
from statistics import mean

datafile = './data/pima-indians-diabetes.csv'

df = pd.read_csv(datafile, header=0, index_col=None)
df.head()

,times_pregnant,plasma_glucose_concentration,diastolic_blood_pressure,triceps_thickness,2-hour_serum_insulin,BMI,diabetes_pedigreen,age,diabetes
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [27]:
# Missing data is coded as 0 in all but the first and last columns (where 0 is meaningful)
# Change 0s in columns 1-7 to np.nan

# Hint: you can conditionally assign slices of a dataframe to a new value:
# eg. df[df[c]==x] = v  will change values in column c to v where they were originall x.
# And remember you can slice multiple columns with as df[[0,1]] for the first two columns
# You can also look at pandas' DataFrame.replace() function

replace_dict = {'plasma_glucose_concentration': 0, 'diastolic_blood_pressure': 0, 'triceps_thickness': 0, '2-hour_serum_insulin': 0, 'BMI': 0, 'diabetes_pedigreen': 0, 'age': 0}

df.replace(replace_dict, np.nan, inplace=True)
#check min values for selected columns no longe 0.0
df.describe()



,times_pregnant,plasma_glucose_concentration,diastolic_blood_pressure,triceps_thickness,2-hour_serum_insulin,BMI,diabetes_pedigreen,age,diabetes
count,768.000000,763.000000,733.000000,541.000000,394.000000,757.000000,768.000000,768.000000,768.000000
mean,3.845052,121.686763,72.405184,29.153420,155.548223,32.457464,0.471876,33.240885,0.348958
std,3.369578,30.535641,12.382158,10.476982,118.775855,6.924988,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,22.000000,76.250000,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,29.000000,125.000000,32.300000,0.372500,29.000000,0.000000
75%,6.000000,141.000000,80.000000,36.000000,190.000000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [4]:
# The basics: counts, min, max, mean, median and mode
# Get the count (of all non-NaNs), min, max, mean, median and mode of each column.

#set index and labels
index = ['Mean', 'Median', 'Min', 'Max', 'Non NaN Values']
labels = ['times_pregnant', 'plasma_glucose_concentration', 'diastolic_blood_pressure', 'triceps_thickness', '2-hour_serum_insulin', 'BMI', 'diabetes_pedigreen', 'age', 'diabetes']

#get all stats values that we can with df built in funcs
means = df.mean(axis=0, skipna=True) #=> describe already skips na
medians = df.median(axis=0, skipna=True)
mins = df.min(axis=0, skipna=True)
maxs = df.max(axis=0, skipna=True)
not_na_counts = df.notnull().sum()

#create stats df
statdf = pd.DataFrame([means, medians, mins, maxs, not_na_counts], index=index)

#get mode using stats.mode function
mode_result = stats.mode(df, nan_policy='omit')
mode_arr = mode_result[0][0]

# create mode df to add to other stats df
modes = pd.DataFrame([[1, 99, 70, 32, 105, 32, 0.254, 22, 0]], columns=labels, index=['Mode'])

#join mode df to stats df
statdf.append(modes)

# Which column has the most missing values?
    # just look at non-null count and find min


,times_pregnant,plasma_glucose_concentration,diastolic_blood_pressure,triceps_thickness,2-hour_serum_insulin,BMI,diabetes_pedigreen,age,diabetes
Mean,3.845052,121.686763,72.405184,29.15342,155.548223,32.457464,0.471876,33.240885,0.348958
Median,3.000000,117.000000,72.000000,29.00000,125.000000,32.300000,0.372500,29.000000,0.000000
Min,0.000000,44.000000,24.000000,7.00000,14.000000,18.200000,0.078000,21.000000,0.000000
Max,17.000000,199.000000,122.000000,99.00000,846.000000,67.100000,2.420000,81.000000,1.000000
Non NaN Values,768.000000,763.000000,733.000000,541.00000,394.000000,757.000000,768.000000,768.000000,768.000000
Mode,1.000000,99.000000,70.000000,32.00000,105.000000,32.000000,0.254000,22.000000,0.000000


In [77]:
# Now let's try to better characterize the column distributions.
# First, we can "centre" each column (also called a z-transform) so that it has mean=0, variance=1
# To center a value, we simply subtract the mean of the distribution, and divide that value 
#   by the standard deviation of the distribution:
# for each value v in a distribution of values V:
#    v = (v-mean(V)) / std(V)

# Write a centre() function
# Hint: centre([0,1,2,3,5,10]) => [-1.059, -0.757, -0.454, -0.151, 0.454, 1.967]

# Remember to use np.nanmean() and np.nanstd() to deal with NaNs

def centre(col):
    mean_col = np.nanmean(col)
    std_col = np.nanstd(col)
    array_vals = col.to_numpy()
    centered_vals = stats.zscore(array_vals, nan_policy='omit')
    return centered_vals

centered_plasma = centre(df['plasma_glucose_concentration'])

def check_z_transform(values):
    values = np.array(values)
    mean = round(np.nanmean(values), 2)
    std = np.nanstd(values)
    print('Mean: ', mean, '\nStd: ', std)

check_z_transform(centered_plasma)




Mean:  0.0 
Std:  1.0


In [83]:

df2 = df.copy()

# print(np.nanmean(values[][1]))
def centre2(df2):
    values = df2.values
    calc = (values - np.nanmean(values, axis=0)/np.nanstd(values, axis=0))
    df3 = pd.DataFrame(calc, columns = labels)
    return df3

def centre3(df2):
    # print(df2.mean(), df2.std())
    return ((df2 - df2.mean())/df2.std())

df3 = centre3(df2).round(2)

df3.head()
df3.describe()
# new_vals = centre2(values)
# # print(new_vals[1][:5])
# # print(values[1][:5])
# df3 = pd.DataFrame(new_vals, columns=labels)
# # print(df3.iloc[1][:5])
# df3.describe()


,times_pregnant,plasma_glucose_concentration,diastolic_blood_pressure,triceps_thickness,2-hour_serum_insulin,BMI,diabetes_pedigreen,age,diabetes
count,768.000000,763.000000,733.000000,541.000000,394.000000,757.000000,768.000000,768.000000,768.000000
mean,0.001003,0.000197,-0.000014,0.000222,-0.000228,-0.000040,-0.000104,-0.001263,0.002813
std,0.999467,0.999918,0.999702,1.000039,1.000088,0.999924,0.999958,1.000932,1.001598
min,-1.140000,-2.540000,-3.910000,-2.110000,-1.190000,-2.060000,-1.190000,-1.040000,-0.730000
25%,-0.840000,-0.740000,-0.680000,-0.680000,-0.667500,-0.720000,-0.690000,-0.790000,-0.730000
50%,-0.250000,-0.150000,-0.030000,-0.010000,-0.260000,-0.020000,-0.300000,-0.360000,-0.730000
75%,0.640000,0.630000,0.610000,0.650000,0.290000,0.600000,0.470000,0.660000,1.370000
max,3.900000,2.530000,4.010000,6.670000,5.810000,5.000000,5.880000,4.060000,1.370000


In [82]:
# Let's make a copy of df, to hold the centred values:
cdf = df.copy()

cdf = cdf.apply(centre)
cdf = cdf.round(2)

cdf.head()
cdf.describe()

,times_pregnant,plasma_glucose_concentration,diastolic_blood_pressure,triceps_thickness,2-hour_serum_insulin,BMI,diabetes_pedigreen,age,diabetes
count,768.000000,763.000000,733.000000,541.000000,394.000000,757.000000,768.000000,768.000000,768.000000
mean,0.001042,0.000079,0.000314,0.000018,-0.000178,0.000079,-0.000065,-0.000586,0.002813
std,0.999596,1.000616,1.000813,1.000781,1.001355,1.000823,1.000559,1.001484,1.001598
min,-1.140000,-2.550000,-3.910000,-2.120000,-1.190000,-2.060000,-1.190000,-1.040000,-0.730000
25%,-0.840000,-0.740000,-0.680000,-0.680000,-0.667500,-0.720000,-0.690000,-0.790000,-0.730000
50%,-0.250000,-0.150000,-0.030000,-0.010000,-0.260000,-0.020000,-0.300000,-0.360000,-0.730000
75%,0.640000,0.630000,0.610000,0.650000,0.290000,0.600000,0.470000,0.660000,1.370000
max,3.910000,2.530000,4.010000,6.670000,5.820000,5.010000,5.880000,4.060000,1.370000


In [24]:
# Finally! Let's look for outliers by getting the min and max in the new centred df:
# (You can re-use code from above!)

mins = cdf.min(axis=0, skipna=True)
maxs = cdf.max(axis=0, skipna=True)
means = cdf.mean(axis=0, skipna=True)
statcdf = pd.DataFrame([mins, maxs, means], index=['Min', 'Max', 'Mean'])
statcdf



,times_pregnant,plasma_glucose_concentration,diastolic_blood_pressure,triceps_thickness,2-hour_serum_insulin,BMI,diabetes_pedigreen,age,diabetes
Min,-1.140000,-2.550000,-3.910000,-2.120000,-1.190000,-2.060000,-1.190000,-1.040000,-0.730000
Max,3.910000,2.530000,4.010000,6.670000,5.820000,5.010000,5.880000,4.060000,1.370000
Mean,0.001042,0.000079,0.000314,0.000018,-0.000178,0.000079,-0.000065,-0.000586,0.002813


In [25]:
# trick question:
# What is the mean of cdf['times_pregnant']?
# Why isn't it zero?

something = [1, 2, 5, 4, 9, 15, 4, 2, 4, 9]
zsomething = stats.zscore(something)
zsomething = np.array(zsomething)

# ???? 
# thought maybe somethig to do with all positive ints but no luck

In [26]:
# need to adjust which columns - did the pregnent column replace value where 0 is a valid vaule

In [86]:
def centre(col):
    array_vals = col.to_numpy()
    return stats.zscore(array_vals, nan_policy='omit')

cdf = df.copy()
cdf = cdf.apply(centre).round(3)
cdf.describe()

,times_pregnant,plasma_glucose_concentration,diastolic_blood_pressure,triceps_thickness,2-hour_serum_insulin,BMI,diabetes_pedigreen,age,diabetes
count,768.000000,763.000000,733.000000,541.000000,394.000000,757.000000,768.000000,768.000000,768.000000
mean,-0.000020,0.000008,0.000018,-0.000002,0.000005,-0.000022,0.000030,0.000046,0.000115
std,1.000765,1.000672,1.000694,1.000935,1.001286,1.000660,1.000651,1.000611,1.000644
min,-1.142000,-2.546000,-3.912000,-2.116000,-1.193000,-2.060000,-1.190000,-1.042000,-0.732000
25%,-0.845000,-0.743000,-0.679000,-0.683000,-0.668750,-0.716000,-0.688750,-0.786000,-0.732000
50%,-0.251000,-0.154000,-0.033000,-0.015000,-0.258000,-0.023000,-0.300500,-0.361000,-0.732000
75%,0.640000,0.633000,0.614000,0.654000,0.290000,0.599000,0.465750,0.660000,1.366000
max,3.907000,2.534000,4.008000,6.673000,5.820000,5.006000,5.884000,4.064000,1.366000
